# Fast injection generation

In [1]:
import sys
sys.path.append('..')

from src.gw.detector import *

Pattern function for different detectors:

In [3]:
for detector in DetectorCoordinates:
    name = detector.name
    coords = detector.value
    position = DetectorPosition(*coords)
    fplus, fcross = position.pattern_function()
    print(f"Pattern functions for {name}: F+ = {fplus:.2f}, Fx = {fcross:.2f}")

Pattern functions for LIGO_HANFORD: F+ = 0.47, Fx = 0.57
Pattern functions for LIGO_LIVINGSTON: F+ = 0.50, Fx = -0.38
Pattern functions for VIRGO: F+ = 0.37, Fx = 0.63
Pattern functions for KAGRA: F+ = 0.42, Fx = 0.42
